In [25]:
from typing import Dict, List, Any, TypedDict, Annotated, Sequence
import operator
import json

from langgraph.graph import StateGraph, END
from langchain_openai import AzureChatOpenAI
from langchain_core.callbacks import BaseCallbackManager, StdOutCallbackHandler
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage, BaseMessage
from langchain_core.utils.function_calling import convert_to_openai_tool
from langgraph.prebuilt import ToolInvocation, ToolExecutor

In [26]:
llm = AzureChatOpenAI(
    model="gpt-35",
    temperature=0.5,
    max_tokens=256,
)

In [27]:
def start_docker(**args):
    "Start Docker container"
    return "STARTING Docker ..."

def stop_docker(container_id="###"):
    "Stop Docker container"
    return f"STOPPING Docker {container_id}"

def fetch_code():
    "Return code retriever results"
    return "CODE Snippet ###"

def exec_shell():
    "Run shell command in shell"
    return "SHELL Command ..."


tools = [start_docker, stop_docker, fetch_code, exec_shell]
functions = [convert_to_openai_tool(t) for t in tools]

In [28]:
functions

[{'type': 'function',
  'function': {'name': 'start_docker',
   'description': 'Start Docker container',
   'parameters': {'type': 'object', 'properties': {}, 'required': []}}},
 {'type': 'function',
  'function': {'name': 'stop_docker',
   'description': 'Stop Docker container',
   'parameters': {'type': 'object', 'properties': {}, 'required': []}}},
 {'type': 'function',
  'function': {'name': 'fetch_code',
   'description': 'Return code retriever results',
   'parameters': {'type': 'object', 'properties': {}, 'required': []}}},
 {'type': 'function',
  'function': {'name': 'exec_shell',
   'description': 'Run shell command in shell',
   'parameters': {'type': 'object', 'properties': {}, 'required': []}}}]

In [29]:
llm.bind_tools(functions);

In [4]:
llm.invoke("Say Hi")

AIMessage(content='Hi there! How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-54068428-2a27-4d69-a961-b795a14d9b3d-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19})

In [147]:
# tool_executor = ToolExecutor(tools)

In [35]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

def should_continue(state):
    last_message = state['messages'][-1]
    
    if last_message.tool_calls:
        return "tools"

    return "end"

In [36]:
def system_prompt(state):
    messages = state['messages']
    sys_msg = SystemMessage("You are a helpful assistant, looking to for the best decision based on the available functions")
    return {"messages": [sys_msg]}

def llm_response(state):
    messages = state['messages']
    response = llm.invoke(messages)
    return {"messages": [response]}

def exec_function(state):
    messages = state['messages']
    last_message = messages[-1] # this has the query we need to send to the tool provided by the agent

    parsed_tool_input = json.loads(last_message.additional_kwargs["function_call"]["arguments"])
    
    print("parsed_tool_input:")
    print("--"*50)
    print(parsed_tool_input)
    print("--"*50)

    # We construct an ToolInvocation from the function_call and pass in the tool name and the expected str input for OpenWeatherMap tool
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=parsed_tool_input['__arg1'],
    )
    
    # # We call the tool_executor and get back a response
    # response = tool_executor.invoke(action)

    # # We use the response to create a FunctionMessage
    # function_message = FunctionMessage(content=str(response), name=action.tool)
    tool_message = ToolMessage(content="None", name=action.tool)

    # We return a list, because this will get added to the existing list
    return {"messages": [tool_message]}

In [37]:
workflow = StateGraph(AgentState)

workflow.add_node("start", system_prompt)
workflow.add_node("llm", llm_response)
workflow.add_node("tools", exec_function)

# workflow.add_edge('start', 'llm')
# workflow.add_edge('llm', END)

workflow.add_edge('start', 'llm')
workflow.add_conditional_edges("llm", should_continue, {"continue": "tools", "end": END})
workflow.add_edge('tools', 'llm')


workflow.set_entry_point("start")
app = workflow.compile()
app.name="isseAgent"

In [38]:
# app.invoke(HumanMessage("get me a code snippet then test it in an isolated environment"))

In [53]:
input = {"messages": [HumanMessage("get me a code snippet then test it in an isolated environment")]}
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value["messages"][0])
    print("\n---\n")

Output from node 'start':
---
content='You are a helpful assistant, looking to for the best decision based on the available functions'

---

Output from node 'llm':
---
content='Here\'s a simple Python code snippet that calculates the factorial of a given number:\n\n```python\ndef factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)\n\n# Test the factorial function\nnumber = 5\nresult = factorial(number)\nprint(f"The factorial of {number} is {result}")\n```\n\nTo test this code in an isolated environment, you can use an online Python compiler or an integrated development environment (IDE) such as PyCharm or Visual Studio Code. Simply copy and paste the code snippet into the environment and run it. You should see the output "The factorial of 5 is 120" printed to the console.' response_metadata={'token_usage': {'completion_tokens': 145, 'prompt_tokens': 40, 'total_tokens': 185}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 

In [ ]:
app.aget_state()